# Linear Generative Models

This notebook explains the background for generative models of data,
explains the assumptions behind linear generative models of data,
and then quickly describes several such models:

- Probabilistic Principal Components Analysis
- Factor Analysis
- $\ell_1$ Sparse Coding
- Spike-and-Slab Sparse Coding

The emphasis is on generating data according to these models,
rather then on describing algorithms for fitting such models to data,
since there is a surfeit of resources for the former
and a surplus of same for the latter.

## Generative Models

A *generative model* of data is a probabilistic model that allows for the generation of samples.

For example, one of the simplest generative models of English is that letters and spaces occur randomly with a frequency calculated by counting letter occurrences over a corpus of text. Our model can be represented by a 27-entry vector composed of the frequencies of each letter.

Here's a sequence generated from such a model,
as first done by Claude Shannon,
father of Information Theory, shortly after World War II:

OCRO HLO RGWR NMIELWIS EU LL NBNESEBYATH EEI ALHENHTTPA OOBTTVA NAH BRL

This looks nothing like English. Anyone who has played Scrabble could've guessed as such: the seven letters one draws from the Scrabble bag look nothing like an English word, usually. As an even more concrete example, we are keenly aware that when a letter "Q" appears, the next letter is highly likely to be a "U".

This might motivate a more complex model, of English as a first-order Markov chain, where the probability of observing a letter is dependent on only the previous letter. Now our model is a $27$ by $27$ matrix, with each row corresponding to the conditional frequency of each letter given the previous letter. This is a more complicated model, and the resulting samples are correspondingly better:

ON IE ANTSOUTINYS ARE T INCTORE ST BE SDEAMY ACHIN D ILONASIVE TUCOOWE ATTEASONARE
FUSO TIZIN ANDY TOBE SEACE CTISBE IN NO IST LAT WHEY CRATICT FROURE BIRS GROCID
PONDENOME OF DEMONSTURES OF THE REPTAGIN IS REGOACTIONA OF CRE

If we extend our model to track the last three letters (aka we model English as a third-order Markov Chain, resulting in a $27$ by $27^3\approx 12000$ matrix), the samples become even better:

THE GENERATED JOB PROVIDUAL BETTER TRAND THE DISPLAYED CODE, ABOVERY UPONDULTS WELL
THE CODERST N THESTICAL IT DO HOCK BOTHE BERG. INSTATES CONS ERATIONS. NEVER ANY OF
PUBLIE AND TO THEORY. ENVTIAL CALLENGAND TO ELAST BENERATED IN WITH PIES AS IS WITH
THE

Kind of looks like someone mixed in some Old English with a typo-ridden technical report.

As we increase the complexity of our model, we generate increasingly plausible samples of the data at the cost of increased computational effort.

In this notebook, we'll look at *factorized linear generative models*, some of the simplest models for continuous random variables -- in some sense, the continuous analogues of the marginal and 1st-order Markov Chains above.

(All English-modeling examples come from [these lecture notes](http://www.cim.mcgill.ca/~langer/423/lecture14.pdf). See them for more details.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

The code in this cell is used to implement the linear generative models below.
It is included for readers interested in implementation details,
but needn't be read for understanding.

In [ ]:
class GenerativeModel(object):
    """
    A generative model has a method for sampling the hidden variables
    and a method for sampling the data, conditioned on the hidden variables.
    
    Also included are methods for plotting the hidden and data distributions
    for the 1 and 2-dimensional cases.
    """
    def __init__(self, conditional_data_sampler, hidden_sampler):
        
        self.hidden_sampler = hidden_sampler
        self.conditional_data_sampler = conditional_data_sampler
        self._has_data = False
        self._has_plot = False
        
    def generate_hidden(self,N):
        hidden_sample = self.hidden_sampler(N)
        return hidden_sample
    
    def generate_data(self,N):
        hidden_sample = self.generate_hidden(N)
        data_sample = self.conditional_data_sampler(hidden_sample)
        self.hidden_sample = hidden_sample
        self.data_sample = data_sample
        self._has_data = True
        self.N = N
    
    def plot(self, which="both"):

        if not self._has_data:
            self.generate_data(50)
        
        if which == "both":
            self.plot(which="data")
            self.plot(which="hidden")
        elif which in ["data", "hidden"]:
            self._plot(which)
        else:
            raise ValueError("which must be data or hidden, but got {0}".format(which))
            
    def _plot(self, which):
        
        if which == "data":
            values = self.data_sample
        elif which == "hidden":
            values = self.hidden_sample
        else:
            raise ValueError("which must be data or hidden, but got {0}".format(which))
        
        if values.shape[0] == 2:
            self._plotgrid = sns.jointplot(*values,
                           kind='scatter', stat_func=None,
                            joint_kws={"alpha":0.05, "s":48}
                          )
            self._plotgrid.ax_joint.set_aspect("equal")
                
        elif values.shape[0] == 1:
            plt.figure()
            sns.distplot(np.squeeze(values), kde=False)
        else:
            raise ValueError("values have shape[0] of {0}, expected 1 or 2".format(values.shape[0]))
            
        self._has_plot = True
        title = which.capitalize()+" Variable Distribution"
        plt.suptitle(title,
                    y=1.1,
                    weight="bold", size="xx-large")
        
    def add_solutions(self):
        assert self._has_plot, "plot not created yet"
        if (hasattr(self,"W")) & (self.W.shape == (2,1)):
            values = self.data_sample
            x_min, x_max = (np.min(values[0]), np.max(values[0]))

            x_range = np.expand_dims(np.linspace(x_min, x_max, num=3),0)
            self._plotgrid.ax_joint.plot(*np.dot(W,x_range),
                                         linewidth=2, color='k',
                                        label = r'$W\cdot h$')

            self._plotgrid.ax_joint.legend()
        else:
            raise NotImplementedError("couldn't determine how to plot solutions")
            
class LinearGenerativeModel(GenerativeModel):
    """
    A linear generative model has a hidden-conditional data sampler
    that is parameterized by a matrix W and a noise-covariance matrix sigma.
    
    For convenience, sigma can also be a scalar, which produces isotropic noise.
    """
    
    def __init__(self, W, sigma, hidden_sampler):
        
        conditional_data_sampler = self._make_linear_sampler(W,sigma)
        GenerativeModel.__init__(self, conditional_data_sampler, hidden_sampler)
        self.W = W
        
    def _make_linear_sampler(self, W, sigma):
        return lambda h: np.dot(W,h) + \
                            np.dot(sigma,
                                   np.random.standard_normal(size=(W.shape[0],h.shape[1])))

## Factorized Linear Generative Models

### Modeling Random Variables

Above, we simplified the challenge of modeling English, a sequence of discrete symbols with unkown structure, by considering models that capture the structure of much simpler sequences: sequences that had no dependence on their past (i.i.d sequences) and then sequences with increasing dependence on their past (Markov chains of increasing order).

How do we simplify the challenge of modeling a continuous random variable?

First, let's define what it means to model a random variable, as opposed to a sequence.

A model of a random variable is a representation of its probability distribution. For example, I might model the random variable "My Location" (which takes values in latitude and longitude) by saying that it is approximately a mixture of two Gaussian distributions, one centered at my workplace and the other at my house.

Written explicitly:

$$
p(\text{lat},\text{long}) \approx 0.5 g_{home}(\text{lat},\text{long}) + 0.5 g_{work}(\text{lat},\text{long})
$$

where $g_{location}$ is a Gaussian distribution centered at $location$.

If I think of "I am at home" and "I am at work" as values of a random variable, then we can rewrite the model above. Using $H$ for the random variable, with $h=1$ corresponding to "I am at home" and $h=0$ corresponding to "I am at work", we get: 

$$
p(\text{lat},\text{long}\lvert H=h) \approx h \cdot g_{home}(\text{lat},\text{long}) + (1-h)\cdot g_{work}(\text{lat},\text{long})
$$

For example, this model says that, if we know that I am at home, then

$$
p(\text{lat},\text{long}\lvert H=1) \approx h \cdot g_{home}(\text{lat},\text{long})
$$

and therefore the chance that my latitude and longitude take on any given value is a decreasing function of the distance from my house.

The utility of this model is that I have simplified what could be a very complicated distribution into something extremely simple -- so simple that we can work with it mathematically, with only a few symbols.
So long as the assumptions of the model aren't too incorrect, we can work directly with the model to guide our decision-making.

If we really believed that this model did a good job of capturing the behavior of human beings,
we could use it to uncover the location of their home and their workplace,
given a dataset of their location data over time.

A model that proposes that the data can be explained by means of unobserved variables is called a *hidden variable model* or a *latent variable model*, because the variables of interest are "hidden" from us, or equivalently are "latent" in the data. It is sometimes called a *factor model*, since it proposes that unknown factors explain the data. Many generative models are hidden variable models, including all the ones we will consider here.

### Simple Hidden Variable Models: Factorial Linear Generative Models

If we are allowed total freedom in creating a hidden variable model,
then the problem becomes uninteresting:
every dataset is perfectly explained by a hidden variable model
where the hidden variable is the identity or index of the datapoint, $i$,
and the conditional distribution is a delta distribution
at the value of datapoint $i$.

Hidden variable models are useful only when they are constrained.
Furthermore, just like the Markov models of English above,
simpler models are usually easier to learn, store, and sample from.
They might even be so simple that we can make analytic, mathematical statements
about their properties!

#### Simplifying Assumption #1 - Factorial Models

The first major restriction we place on our hidden variables
is that they are *independent* of one another.
That is,

$$
p\left(\vec{h}\right) = \prod_i p\left(h_i\right)
$$

or, in terms of entropies:

$$
H\left(\vec{h}\right) = \sum_i H\left(h_i\right)
$$

In this case, sampling from the model becomes easier because
the hidden variables can be sampled in parallel from their marginal distributions,
rather than sequentially from their joint distribution.
Such a model is called a *factorial* model,
since the joint probability *factorizes* over,
or turns into a product of,
the marginal distributions.
(This issue didn't come up in the "location" model above because
the hidden variable was one dimensional).

Relatedly,
we also must assume that,
once the hidden variables are known,
there is no more unknown structure in the observed variables.

If there were,
then it'd be impossible to know which relationships between observed variables
were due to this unknown structure in the observed variables
and which were due to the unknown structure in the dependence
of the observed variables on the hidden variables.

One way to formalize this is to say that the
observed variables are *conditionally independent of each other*
when conditioned on the hidden variables.
In terms of the distribution,
this condition becomes:

$$
p\left(\vec{x}\lvert\vec{h}\right) = \prod_i p\left(x_i\lvert\vec{h}\right)
$$

or, in terms of conditional entropies

$$
H\left(\vec{x} \lvert \vec{h}\right) = \sum_i H\left(x_i\lvert\vec{h}\right)
$$

Combining this conditional distribution of the data
with the assumed distribution of the hidden variables,
we arrive at an expression for the joint distribution:

$$
p\left(\vec{x},\vec{h}\right) = \prod_i p\left(x_i\lvert\vec{h}\right) \prod_i p(h_i)
$$

or

$$
H\left(\vec{x}, \vec{h}\right) = \sum_i H\left(x_i\lvert \vec{h}\right) + \sum_i H\left(h_i\right)
$$

Graphical models provide an elegant and intuitive visualization
of joint distributions in terms of their conditional independences,
or equivalently in terms of which information measures are 0.
They are exceedingly useful for reasoning about algorithms
for statistical inference and learning,
just as
[computational graphs](http://colah.github.io/posts/2015-08-Backprop/)
are invaluable for reasoning about neural network architectures.
See
[this monograph](https://people.eecs.berkeley.edu/~wainwrig/Papers/WaiJor08_FTML.pdf)
for a thorough introduction
or [this blog post](http://charlesfrye.github.io/stats/2017/09/26/discrete-channel-graph-model.html)
for a much shorter introduction in the context of information theory.

Roughly and in short,
we represent the joint probability distribution by a graph
whose nodes represent individual random variables
and whose edges represent direct dependences between random variables.

For a factorial hidden variable model
with conditionally independent observed variables,
the graphical model is as below,
reproduced from
[Deep Learning](http://www.deeplearningbook.org/),
by Goodfellow, Courville, and Bengio.

![factorized_hidden_variable_model](./factorized_hidden_variable_model.png)

Our assumption that the $h$s are independent of each other
is manifest in the lack of edges between $h$ nodes,
while the assumption that the $x$s are conditionally independent
of each other, given the $h$s,
is manifest in the presence of edges from each $h$ to each $x$
and the absence of edges between $x$s.

The nature of the dependence between the $x$s and $h$s,
represented by the edges connecting those nodes,
is left unspecified.

Our next two simplifying assumptions will resolve this. 

#### Simplifying Assumption #2 - Conditionally Gaussian

In the model of my location from above,
we assumed that knowing the hidden variable
would only reveal my location up to a Gaussian-distributed error.

For a variety of reasons,
the choice of conditional Gaussianity is a decent one.
Briefly:
- **From a Bayesian perspective**: 
we usually describe our errors as having an average value (ideally 0)
and some finite spread.
If we think of our errors that way,
the principle of maximum entropy guides us to
choose a Gaussian distribution to model them.
- **From a classical perspective**:
the effects that give rise to our errors are,
we assume, legion but individually minute and unrelated to each other.
If that's the case, then the Central Limit Theorem
states that they must be distributed approximately as a Gaussian.

Gaussians also possess of numerous fabulous properties
that make calculating interesting quantities related to them
a breeze, which is important for creating a simple model.

Mathematically, this assumption is:

$$
p(\vec{x}\lvert \vec{h}) \propto \mathrm{e}^{-\frac{1}{2} \left(\vec{x}-f\left(\vec{h}\right)\right)^T \Sigma^{-1} \left(\vec{x}-f\left(\vec{h}\right)\right)}
$$

or, more "simply"

$$
-\log p\left(\vec{x}\lvert \vec{h}\right) = -\frac{1}{2}
\left(\vec{x}-f\left(\vec{h}\right)\right)^T \Sigma^{-1} \left(\vec{x}-f\left(\vec{h}\right)\right) + A(\Sigma)
$$

where $f\left(\vec{h}\right)$ is a function that takes in the hidden variables and predicts values of the observed variables, $\Sigma$ is the covariance matrix of the errors induced by using $f\left(\vec{h}\right)$ to predict $\vec{x}$, and $A(\Sigma)$ is a normalizing constant also known as the log-partition function.

The expression $\vec{z}^T P \vec{z}$ is a common one. If $P$ is a positive semi-definite matrix, as all covariance matrices are, then it represents a kind of "distance measure" on the vector $\vec{z}$. For example, if $P$ is $I$, then we have
$$\vec{z}^T I \vec{z} = \vec{z}^T \vec{z} = \sum_i z_i^2 $$

or the squared length of $\vec{z}$, aka the squared distance of the tip of the vector from the origin. Different choices of $P$ result in different "tilted" versions of the squared Euclidean distance, called *Mahalanobis distances*.
See [Wikipedia](https://en.wikipedia.org/wiki/Mahalanobis_distance) for more.

Interpreting $-\log p(x)$
as the *surprise*
associated with observing the value $x$
for a random variable with distribution $p$,
as one should
(see [this blog post](http://charlesfrye.github.io/stats/2016/03/29/info-theory-surprise-entropy.html) for details),
we can read this equation:

$$
-\log p\left(\vec{x}\lvert \vec{h}\right) = -\frac{1}{2}
\left(\vec{x}-f\left(\vec{h}\right)\right)^T \Sigma^{-1} \left(\vec{x}-f\left(\vec{h}\right)\right) + A(\Sigma)
$$

in English as "it is less suprising the find the variable closer to the value predicted from the hidden variable",
and the utility and simplicity of the Gaussian assumption is hopefully more clear.

*Nota bene*:
We have assumed already that the data values,
$\vec{x}$, are conditionally independent given the hidden variables.
For Gaussians, independence is synonymous with decorrelation,
meaning that the off-diagonal elements of $\Sigma$
(and hence, $\Sigma^{-1}$)
must be $0$.
That is, $\Sigma$ is a *diagonal matrix*.

So far,
we have passed over the function $f$ with no comment.
Our final simplifying assumption tackles this piece
and completes our description of the relationship between $\vec{h}$ and $\vec{x}$.

#### Simplifying Assumption #3 - Linearity

If you're looking for a simple class of functions,
one of the first places to look is at the class of *linear* functions.

Every linear function of a vector that produces a scalar
can be represented as a vector,
and application of a linear function to a vector
is just vector-vector multiplication, as in

$$
f\left(\vec{x}\right) = \vec{v}^T\vec{x}
$$

for some $\vec{v}$.

Linear functions of vectors that produce vectors can be represented with matrices,
which are just stacks of linear functions that each produce scalars
(aka row vectors),
one for each element of the output vector,
and the application of such a linear function
is just matrix-vector multiplication.

A matrix is, traditionally,
represented with a capital letter,
so a linear function $f$
of a vector $\vec{x}$
that is represented by a matrix $W$
can be written

$$
f(\vec{x}) = W\vec{x}
$$

This simple relationship between functions and input to functions is one of the sources of the power and elegance of linear algebra.
Entire treatises could be written on the meaning and purpose of linearity,
but we will pass over them and just note that linear functions are simple and play nicely with Gaussians.

With our linear assumption in hand,
we can write the conditional distribution as

$$
-\log p\left(\vec{x}\lvert \vec{h}\right) = -\frac{1}{2}
\left(\vec{x}-W\vec{h}\right)^T \Sigma^{-1} \left(\vec{x}-W\vec{h}\right) + A(\Sigma)
$$

where $W$ is a matrix relating the hidden and observed variables.
Becaues $W$ (linearly) combines together the hidden variables to produce the observed variables,
it is also called the *mixing matrix*.
In sparse coding, to be described below,
it is called the *dictionary*.

This expression,
combined with our graphical model,
is sufficient to describe a family of useful linear generative models.
Despite the strictness of our assumptions,
the resulting models are already sufficiently complex
to necessitate careful analysis.

### Quick Note on Implementation

The cells below will generate data from linear generative models.

The code to generate data is organized into a class,
`LinearGenerativeModel`,
that requires the user to provide

- a mixing matrix
- a conditional variance, either a scalar or a (diagonal) matrix
- a method for sampling the hidden variables
that takes as an argument the number of samples to be generated

It will generates `N` samples from that model
when the method `.generate_data` is called with argument `N`
and plot the distribution of the sampled data variables
(jointly and marginally)
and hidden variable or variables (jointly and marginally)
when the method `.plot("both")` is called.

Calling `.add_solutions`
will plot $Wh$ for a one-dimensional hidden variable.
This is the solution that a proper implementation of the algorithm should find,
provided it has enough data.

### Fully-Gaussian Models

To develop our first set of models,
we add an additional simplifying assumption:
the hidden variables are also Gaussian distributed.

Since linear combinations of Gaussians
(aka the result of linear functions applied to Gaussian random variables)
are themselves Gaussian,
the marginal distributions of the $x$s are Gaussian,
and so the linear generative models
generate Gaussian-distributed data.

#### Probabilistic PCA

If we assume that all the elements along the diagonal are equal to some value $\sigma^2$:

$$
\Sigma = \text{diag}(\sigma^2)
$$

then the resulting generative model is called *Probabilistic Principal Components Analysis*.
$\text{diag}$, here, like `np.diag`, takes in a value or vector of values and returns the matrix
with that value or values along the diagonal.

In [ ]:
W = np.asarray([[0.5],
                [1.2]])

hidden_variable_dimension = W.shape[1]

hidden_variance = np.eye(W.shape[1])

hidden_sampler = lambda N: np.dot(hidden_variance,
                                  np.random.standard_normal(size=(hidden_variable_dimension,N)))

PPCA = LinearGenerativeModel(W, 0.5, hidden_sampler)

In [ ]:
PPCA.generate_data(5000)

In [ ]:
PPCA.plot("both")
PPCA.add_solutions()

If you're familiar with Principal Components Analysis,
the line indicated by $Wh$ should look familiar:
it is exactly the principal eigenvector of the covariance matrix of the data.

However, probabilistic PCA is a generative model,
while PCA is simply a dimensionality reduction technique.
The advantage of PCA over pPCA is that its agnosticism
towards the distribution of the data makes it robust
to variations in that distribution.
pPCA, on the other hand,
will provide incorrect answers for data distributed differently than expected.
The disadvantage of PCA is that it will give incorrect answers
for data actually generated according to the pPCA model.

#### Factor Analysis

A closely-related technique to pPCA is
*Factor Analysis*.
Factor analysis is also a linear generative model
with Gaussian-distributed hidden variables,
but in this model the conditional, or nuisance, variance
$\Sigma$
has possibly different values along the diagonal.

The result is that, for a fixed $h$,
the values of $x$ are distributed
in an ellipse that is axis-aligned.
To see this,
simply run the code below with the
`hidden_variance` set to some small value, like `0.1`.
It might be instructive to compare to the same
for pPCA.

In [ ]:
W = np.asarray([[0.5],
                [1.2]])

hidden_variable_dimension = W.shape[1]

hidden_variance = np.eye(W.shape[1])

hidden_sampler = lambda N: np.dot(hidden_variance,
                                  np.random.standard_normal(size=(hidden_variable_dimension,N)))

sigma = np.asarray([[0.5,0],[0,1]])

FA = LinearGenerativeModel(W, sigma, hidden_sampler)

In [ ]:
FA.generate_data(10000)

FA.plot("both")
FA.add_solutions()

Because pPCA and Factor Analysis have different underlying models of the data,
they will find different solutions on the same data.
See [this set of lecture slides by Geoff Hinton](https://www.cs.toronto.edu/~hinton/csc2515/notes/lec7middle.pdf)
for more.

In short,
because Factor Analysis presumes that noise acts along the individual axes of $x$,
it discounts variability that is only in the direction of one of the $x$ variables,
i.e. high variability in one measurement that doesn't predict variability in other measurements.
This can sometimes be desirable and sometimes indesirable.

### Non-Gaussian Models: Sparse Coding

In sparse coding models,
we allow our distribution of
hidden variables to be non-Gaussian --
specifically,
we aim for the hidden variables to be *sparse*,
in that most of the time,
a given hidden variable is exactly or approximately 0.

Sparse coding was originally proposed
as a model of the behavior of visual neurons.
See [this blog post](http://charlesfrye.github.io/FoundationalNeuroscience/48/)
for more.

#### $\ell_1$ Sparse Coding

In $\ell_1$ sparse coding, the hidden variables are Laplace-distributed.

The Laplace distribution somewhat resembles the Gaussian distribution:

$$
p(x) \propto \mathrm{e}^{-\lambda\lvert x - \mu\rvert}
$$

Just as the log-probability view of the Gaussian revealed a connection to the Euclidean distance,
the log-probability view of the Laplace distribution is illuminating:

$$
-\log p(x) = \lvert x-\mu\rvert+ A(\lambda)
$$

That is, it is more suprising to find $x$ further a way from some position $\mu$,
where now distance is measured using the
[Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry),
also known as the $\ell_1$ norm,
hence the name for $\ell_1$ sparse coding.

*Nota bene*: the similarity of form is due to the fact that both the Gaussian family of distributions and the Laplace family of distributions are examples of
[exponential families](https://en.wikipedia.org/wiki/Exponential_family).
These families,
which are among the only ones which admit
[sufficient statistics](https://en.wikipedia.org/wiki/Sufficient_statistic),
are also known as *log-linear* families.
They play very nicely with graphical models,
as expounded upon in
[Wainwright and Jordan's monograph](https://people.eecs.berkeley.edu/~wainwrig/Papers/WaiJor08_FTML.pdf).

In [ ]:
W = np.asarray([[1],[2.5]])
hidden_sampler = lambda N: np.random.laplace(size=(1,N))

SC = LinearGenerativeModel(W,1,hidden_sampler)

In [ ]:
SC.generate_data(1000)

In [ ]:
SC.plot("both")
SC.add_solutions()

#### Spike-And-Slab Sparse Coding

Though convenient for numerous reasons related to its log-linearity,
the Laplace distribution does not generate hidden variables that are sparse
in the sense that their values are often $0$.
In fact,
since the Laplace distribution is non-atomic,
the probability that a variable with that distribution
takes on any particular value is $0$.

To rectify this,
we can adjust our distribution of each hidden variable
to be a mixture distribution:
with some probabiliy,
the variable is exactly $0$,
and with some probability,
the variable is drawn from some other distribution,
typically Gaussian.

The resulting model is called
*spike-and-slab sparse coding*,
due to the characteristic appearance of its
hidden variable distribution,
as clearly visible below.

In [ ]:
W = np.asarray([[1],[2.5]])

def spike_and_slab_sampler(N, sparsity=0.3):
    switches = np.random.uniform(low=0.5,high=1.5,size=N).astype(int)
    samples = np.zeros((1,N))
    
    for idx,switch in enumerate(switches):
        if switch:
            samples[:,idx] = np.random.standard_normal(size=(1,))
    
    return samples

SSSC = LinearGenerativeModel(W,1,spike_and_slab_sampler)

In [ ]:
SSSC.generate_data(1000)

SSSC.plot("both")
SSSC.add_solutions()